# Library Import and files

- Files available here: https://amazon-reviews-2023.github.io/

In [1]:
import pandas as pd


# HuggingFace Dataset

In [ ]:
# from datasets import load_dataset

# meta_ds = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Automotive", split="full", trust_remote_code=True)

# meta_df = meta_ds.to_pandas() 
# print(df.head())

# Local File

In [2]:

meta_file_path='meta_Automotive.jsonl'


meta_df = pd.read_json(meta_file_path, lines=True)

display(meta_df.head())


,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Automotive,Caltric 3 Pack Oil Filter Compatible with Suzu...,5.0,1,[],[Compatible with Suzuki modelsGSX1100E 1100 19...,13.0,[{'thumb': 'https://m.media-amazon.com/images/...,[],Caltric,"[Automotive, Motorcycle & Powersports, Parts, ...","{'Manufacturer': 'Caltric', 'Brand': 'Caltric'...",B00J9I1BD4,NaN,NaN,NaN
1,Automotive,APL 24 14x1.5 Acorn Spline Wheel Lug Nuts Blac...,4.4,59,"[Note: Fit for aftermarket wheels only, not fi...","[Type : Long Spline, Closed End Thread Pitch ...",None,[{'thumb': 'https://m.media-amazon.com/images/...,[],APL,"[Automotive, Tires & Wheels, Accessories & Par...","{'Material': 'Steel', 'Fastener Type': 'Lock',...",B076PZ9X5R,NaN,NaN,NaN
2,Automotive,Vent Cap,5.0,2,[],[030156-40 Features: -Use for power unit canis...,None,[{'thumb': 'https://m.media-amazon.com/images/...,[],Unknown,[],"{'Manufacturer': 'Eureka', 'Brand': 'Unknown',...",B00CMEHS34,NaN,NaN,NaN
3,Automotive,OkuTech Black Retractable Cargo Cover Rear Car...,3.0,1,[Black Retractable SUV Cargo Security Shade Co...,"[Product Details:, Fits Car Model: Dodge Journ...",139.99,[{'thumb': 'https://m.media-amazon.com/images/...,[],OkuTech,"[Automotive, Interior Accessories, Consoles & ...","{'Manufacturer': 'Okutech', 'Brand': 'OkuTech'...",B09MWGTYL3,NaN,NaN,NaN
4,Automotive,Cardone Service Plus 14-1090 Remanufactured Ca...,4.8,5,[Meets or exceeds OE performance and original ...,"[At Cardone, quality is the foundation of our ...",35.76,[{'thumb': 'https://m.media-amazon.com/images/...,[],Amazon Renewed,"[Automotive, Replacement Parts, Brake System, ...","{'Manufacturer': 'Cardone Service Plus', 'Bran...",B009EJV2Y6,NaN,NaN,NaN


In [3]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003129 entries, 0 to 2003128
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   main_category    object 
 1   title            object 
 2   average_rating   float64
 3   rating_number    int64  
 4   features         object 
 5   description      object 
 6   price            object 
 7   images           object 
 8   videos           object 
 9   store            object 
 10  categories       object 
 11  details          object 
 12  parent_asin      object 
 13  bought_together  float64
 14  subtitle         object 
 15  author           object 
dtypes: float64(2), int64(1), object(13)
memory usage: 244.5+ MB


In [4]:
print(meta_df['parent_asin'].nunique())

2003129


# How many categories?

In [5]:
meta_df['path_length'] = meta_df['categories'].apply(len)

max_path_length = meta_df['path_length'].max()

print(f"Maximum path length: {max_path_length}")


Maximum path length: 9


## split df categories to then join with original df

In [9]:
# Step 1: Split the 'categories' column into 9 columns
df_split = meta_df['categories'].apply(lambda x: ','.join(x)).str.split(',', expand=True)

# Step 2: Rename columns (e.g., category_1, category_2, ..., category_9)
df_split.columns = [f'category_{i+1}' for i in range(df_split.shape[1])]


# Encode missing values as a new category, with page path number
for col in df_split.columns:
    df_split[col] = df_split[col].fillna(meta_df['path_length'])


# Step 3: Concatenate the new split columns back to the original DataFrame (meta_df)
meta_df = pd.concat([meta_df, df_split], axis=1)


meta_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2003129 entries, 0 to 2003128
Data columns (total 35 columns):
 #   Column           Dtype  
---  ------           -----  
 0   main_category    object 
 1   title            object 
 2   average_rating   float64
 3   rating_number    int64  
 4   features         object 
 5   description      object 
 6   price            object 
 7   images           object 
 8   videos           object 
 9   store            object 
 10  categories       object 
 11  details          object 
 12  parent_asin      object 
 13  bought_together  float64
 14  subtitle         object 
 15  author           object 
 16  path_length      int64  
 17  category_1       object 
 18  category_2       object 
 19  category_3       object 
 20  category_4       object 
 21  category_5       object 
 22  category_6       object 
 23  category_7       object 
 24  category_8       object 
 25  category_9       object 
 26  category_1       object 
 27  category_2  

In [7]:
print(meta_df.isnull().sum())


main_category        34116
title                    0
average_rating           0
rating_number            0
features                 0
description              0
price               871074
images                   0
videos                   0
store                18820
categories               0
details                  0
parent_asin              0
bought_together    2003129
subtitle           2002955
author             2003069
path_length              0
category_1               0
category_2               0
category_3               0
category_4               0
category_5               0
category_6               0
category_7               0
category_8               0
category_9               0
dtype: int64


# Image and video count

In [10]:
meta_df['num_images'] = meta_df['images'].apply(len)

meta_df['num_videos'] = meta_df['videos'].apply(len)


# Feature string parsing

## feature df

In [67]:
# generate random row sample to get values from dict
print(meta_df['details'].sample(n=1).iloc[0])


{'Brand': 'Dorman', 'Style': 'Compatible', 'Number of Pieces': '1', 'Exterior Finish': 'Machined', 'Handle Type': 'Knob, Lever', 'Special Feature': 'Easy to Install', 'Included Components': 'Dorman 83655: Interior Door Handle Front/Rear Left', 'Finish Type': 'Painted', 'Unit Count': '1.0 Count', 'Number of Items': '1', 'Manufacturer': 'Dorman Products', 'Model': '83655', 'Item Weight': '0.353 ounces', 'Product Dimensions': '6.69 x 3.58 x 1.97 inches', 'Country of Origin': 'Taiwan', 'Item model number': '83655', 'Manufacturer Part Number': '83655', 'OEM Part Number': '83655; 6920647010C0; 6920648010C0; 6920660100C0', 'Special Features': 'Easy to Install', 'Domestic Shipping': 'Item can be shipped within U.S.', 'International Shipping': 'This item can be shipped to select countries outside of the U.S.  Learn More', 'Date First Available': 'October 7, 2013'}


In [74]:
df_feature = meta_df[['title','details','parent_asin']]

In [75]:
display(df_feature.sample(n=10))

,title,details,parent_asin
331769,Wall Stickz Auto Sport 2PCS LED Cup Holder Mat...,"{'Manufacturer': 'Hisport', 'Brand': 'AutoSpor...",B07TYYFQMV
1416763,GIVI PL3112CAM Saddlebag Holder for Trekker Ou...,"{'Manufacturer': 'GIVI', 'Brand': 'GIVI', 'Mod...",B074XBPQ9N
395771,CLYMER B740 MERCRUISER STERN DRIVE SHOP MANUAL...,"{'Brand': 'Clymer - Orange Cycle Parts', 'Date...",B004X6X0XQ
646532,Zenex HID Kit Single Beam Wire Relay Harness (...,"{'Brand': 'Zenex', 'Material': 'Polyvinyl Chlo...",B00GEIIPZ4
1917925,NEH Motorcycle Bike Cover Travel Dust Storage ...,"{'Manufacturer': 'KapscoMoto', 'Brand': 'North...",B0170O6X72
1930440,RUIYA Center Touch Control Screen Protector fo...,{'Package Dimensions': '10.59 x 6.42 x 0.71 in...,B08R3TJJS9
1924165,RYC New AC Compressor and A/C Clutch GH664 (Do...,"{'Manufacturer': 'RY AC Compressors', 'Brand':...",B08WKN8MGT
1922156,Garage-Pro Auto Body Repair Kit Compatible wit...,"{'Material': 'Plastic', 'Brand': 'Garage-Pro',...",B07GVBFDBW
1254533,LIBERTY GRAND CHEROKEE COMMANDER PURGE VALVE P...,"{'Brand': 'Mopar', 'Item Dimensions LxWxH': '8...",B007EZ8GR8
115778,Joes Racing Products 34358 LW Aluminum Quick C...,"{'Material': 'Aluminum', 'Metal Type': 'Alumin...",B01N7BJ8OO


In [79]:
# unique_keys = set()
# for details_dict in df_feature['details']:
#     if isinstance(details_dict, dict):  # Ensure it's a dictionary
#         unique_keys.update(details_dict.keys())

# # Display the unique keys
# print(unique_keys)

{'', 'Voltage 71 Volts', 'Material Natural Rubber, Silicone', 'Power Source 🅰🆄🆃🅴🅻 5X Faster, 11,600mAh Battery Powered 🚀Bigger 4+128GB 🚀 Newest Android 10 🚀Powerful Exynos 8895 Octa-core OS 🚀Stable V200 VCI 🚀8.0-inch 1920*1200 LCD screen🅰🆄🆃🅴🅻 5X Faster, 11,600mAh Battery Powered 🚀Bigger 4+128GB 🚀 Newest Android 10 🚀Powerful Exynos 8895 Octa-co…  See more', 'Style Extended Girder Shift Lever', 'Vehicle Service Type All of the above', 'Exterior Finish Aluminum,Abs Plastic,Polished,Plastic', 'Vehicle Service Type ATV, Scooter, Cruiser/ Touring, UTV, Street Bike, Snowmobile, Personal WatercraftATV, Scooter, Cruiser/ Touring, UTV, Street Bike, Snowmobile, Personal Watercraft  See more', 'Finish Type Brushed', 'Exterior Finish Aluminum,Chrome,Abs,Plastic,Silver', 'Item Weight 2.24 Ounces', 'Power Source Gas-powered', 'Vehicle Service Type Scooter, Cruiser/ Touring, All of the above, Street Bike, SnowmobileScooter, Cruiser/ Touring, All of the above, Street Bike, Snowmobile  See more', 'Item 

In [76]:
keys_to_extract = [
    'Brand',
    'Color',
    'Item Weight',
    'Package Dimensions',
    'Is Discontinued By Manufacturer',
    'Manufacturer Part Number',
    'OEM Part Number',
    'Date First Available'
    'Number of Pieces',
    'Exterior Finish',
    'Handle Type',
    'Special Feature',
    'Included Components',
]


# Loop over the list of keys and create new columns in the DataFrame
for key in keys_to_extract:
    column_name = key.replace(' ', '_').replace('By', '_By')  # Replace spaces and handle "By" for column names
    df_feature[column_name] = df_feature['details'].apply(lambda x: x.get(key, None))


# df_feature['Brand'] = df_feature['details'].apply(lambda x: x.get('Brand', None))
# df_feature['Color'] = df_feature['details'].apply(lambda x: x.get('Color', None))
# df_feature['Item_Weight'] = df_feature['details'].apply(lambda x: x.get('Item Weight', None))
# df_feature['Package_Dimensions'] = df_feature['details'].apply(lambda x: x.get('Package Dimensions', None))
# df_feature['Is_Discontinued'] = df_feature['details'].apply(lambda x: x.get('Is Discontinued By Manufacturer', None))
# df_feature['Manufacturer_Part_Number'] = df_feature['details'].apply(lambda x: x.get('Manufacturer Part Number', None))
# df_feature['OEM_Part_Number'] = df_feature['details'].apply(lambda x: x.get('OEM Part Number', None))
# df_feature['Date_First_Available'] = df_feature['details'].apply(lambda x: x.get('Date First Available', None))

# Number of Pieces
# Exterior Finish
# Handle Type
# Special Feature
# Included Components

C:\Users\nosta\AppData\Local\Temp\ipykernel_22900\2315687258.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature[column_name] = df_feature['details'].apply(lambda x: x.get(key, None))
C:\Users\nosta\AppData\Local\Temp\ipykernel_22900\2315687258.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feature[column_name] = df_feature['details'].apply(lambda x: x.get(key, None))
C:\Users\nosta\AppData\Local\Temp\ipykernel_22900\2315687258.py:21: SettingWithCopyWarning: 
A value is trying to be set

In [77]:
display(df_feature.sample(n=10))

,title,details,parent_asin,Brand,Color,Item_Weight,Package_Dimensions,Is_Discontinued__By_Manufacturer,Manufacturer_Part_Number,OEM_Part_Number,Date_First_AvailableNumber_of_Pieces,Exterior_Finish,Handle_Type,Special_Feature,Included_Components
318510,ECCPP Universal Carbon Fibre Spoiler Wing Rear...,"{'Manufacturer': 'ECCPP', 'Brand': 'ECCPP', 'M...",B085DD15G2,ECCPP,As Shown,1.65 pounds,None,None,117505-5211-1550364794,None,None,None,None,None,None
1799937,Motosport Alloys (16pk) Spike Tapered Lug Nut ...,"{'Manufacturer': 'Motosport Alloys', 'Brand': ...",B078HFKJ5Y,MotoSport Alloys,None,None,None,None,MA2917,None,None,None,None,None,None
34900,"Mitsubishi MN137117, Engine Valve Cover Gasket","{'Manufacturer': 'Mitsubishi', 'Brand': 'Mitsu...",B00736D7GI,Mitsubishi,None,None,None,None,MN137117,MN137117,None,None,None,None,None
1827189,Rock Stopperz Door Hinge Protection kit - Comp...,"{'Brand': 'Rock Stopperz', 'Item Dimensions Lx...",B09Q5WDC4G,Rock Stopperz,None,1.27 ounces,None,None,None,None,None,None,None,Transparent,None
178612,Intro-Tech Automotive TT-84-P Custom Fit Premi...,"{'Brand': 'Intro-Tech Automotive', 'Color': 'S...",B07TBT9NBV,Intro-Tech Automotive,Silver,5 pounds,None,None,TT-84-P,None,None,None,None,None,None
1962589,Fly Racing 2019 Lite Gloves (Large) (Blue/White),{'Item Package Dimensions L x W x H': '13.27 x...,B07D5CRRSY,Fly Racing,None,None,None,None,None,None,None,None,None,None,None
1085302,SAVE THE PLANET It's the only one with WINE - ...,"{'Brand': 'Barefoot Graphix', 'Color': 'Clear,...",B00N19P5NE,Barefoot Graphix,"Clear,white",None,None,None,Bf-a-1978,None,None,None,None,Weatherproof,None
571103,"WheelGuard 1964L, Bulge Acorn Lug Nut, Thread ...","{'Material': 'Steel', 'Fastener Type': 'Lock',...",B06XZF1JQV,WheelGuard,None,5 ounces,None,No,05010123,None,None,Chrome,None,None,None
99131,SSBC W111-2 At The Wheels Rear Disc Brake Conv...,"{'Manufacturer': 'SSBC', 'Brand': 'SSBC', 'Ite...",B001PZYKGY,SSBC,None,55 pounds,None,None,W1112,None,None,None,None,None,None
1946022,Keystone 51711977047 FITS BMW 3 SERIES M3 FRON...,"{'Manufacturer': 'KEYSTONE', 'Brand': 'KEYSTON...",B00O2D4Q8S,KEYSTONE,Black,None,None,None,51711977047,None,None,Smooth,None,None,None


In [78]:
print(df_feature.isnull().sum())


title                                         0
details                                       0
parent_asin                                   0
Brand                                    172665
Color                                   1486825
Item_Weight                              451712
Package_Dimensions                      1403517
Is_Discontinued__By_Manufacturer        1340506
Manufacturer_Part_Number                 372717
OEM_Part_Number                         1524622
Date_First_AvailableNumber_of_Pieces    2003129
Exterior_Finish                         1867348
Handle_Type                             1992939
Special_Feature                         1840607
Included_Components                     1947689
dtype: int64
